In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import numpy as np
from matplotlib import pyplot as plt

from dlf.sequential import Sequential
from dlf.layers.dense import Dense
from dlf.activation_functions.tanh import TanH
from dlf.activation_functions.relu import ReLU
from dlf.regularization.dropout import Dropout
from dlf.optimizers.adam import AdaptiveMomentEstimation
from dlf.normalization.batchnorm import BatchNormalization
from dlf.trainer import Trainer
from dlf.normalization.layernorm import LayerNormalization
from dlf.losses.cce import CategoricalCrossEntropy

from dlf.utils.dataset import train_test_split

In [3]:
m, n = 2000, 4
inputs = np.random.uniform(-1, 1, size=(m, n))
y = (inputs > 0).sum(axis=1)

labels = np.zeros((m, n + 1))
labels[range(m), y] = 1

In [4]:
training_inputs, training_labels, testing_inputs, testing_labels = train_test_split(inputs, labels, 0.7)
testing_inputs, testing_labels, validation_inputs, validation_labels = train_test_split(testing_inputs, testing_labels, 0.5)

In [5]:
print(training_inputs.shape, training_labels.shape)
print(validation_inputs.shape, validation_labels.shape)
print(testing_inputs.shape, testing_labels.shape)

(1400, 4) (1400, 5)
(300, 4) (300, 5)
(300, 4) (300, 5)


In [6]:
training_set = (training_inputs, training_labels)
validation_set = (validation_inputs, validation_labels)
testing_set = (testing_inputs, testing_labels)

In [7]:
model = Sequential([
    Dense(n, 8),
    ReLU(),
    BatchNormalization(),
    Dense(8, 16),
    ReLU(),
    BatchNormalization(),
    Dropout(keep_prob=0.3),
    Dense(16, 32),
    TanH(),
    LayerNormalization(),
    Dropout(keep_prob=0.4),
    Dense(32, n + 1)
])

In [8]:
loss = CategoricalCrossEntropy()

In [9]:
n_epochs = 100
learning_rate = 0.1
batch_size = 128
beta1 = 0.9
beta2 = 0.99

In [10]:
optimizer = AdaptiveMomentEstimation(learning_rate, beta1, beta2)

In [11]:
trainer = Trainer(model, optimizer, loss)

In [12]:
trainer.fit(training_set, n_epochs, batch_size)

[1/100]: Training Loss = 2.325544569453949
[2/100]: Training Loss = 1.6660884902368764
[3/100]: Training Loss = 1.5993710486929589
[4/100]: Training Loss = 1.585134600522237
[5/100]: Training Loss = 1.5451409301817414
[6/100]: Training Loss = 1.5390186928820087
[7/100]: Training Loss = 1.5227401714814714
[8/100]: Training Loss = 1.5274712012175506
[9/100]: Training Loss = 1.5157808713321241
[10/100]: Training Loss = 1.4990774608083983
[11/100]: Training Loss = 1.5038986420003064
[12/100]: Training Loss = 1.4873187656530493
[13/100]: Training Loss = 1.4809809124146998
[14/100]: Training Loss = 1.4694016327998942
[15/100]: Training Loss = 1.454376725549564
[16/100]: Training Loss = 1.4323197114792385
[17/100]: Training Loss = 1.4329014919228282
[18/100]: Training Loss = 1.4397400321925422
[19/100]: Training Loss = 1.4188215948037208
[20/100]: Training Loss = 1.445575285259874
[21/100]: Training Loss = 1.4392080042829611
[22/100]: Training Loss = 1.4191923059115026
[23/100]: Training Loss